## Import the necessary Packages

In [2]:
from dataclasses import dataclass
from typing import Optional

from langchain_ollama import OllamaLLM, ChatOllama, OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

## LLM Setup

In [3]:
@dataclass(frozen=True)
class ModelRegistry:
    # LLMs
    TINY_LLAMA: str = "llama3.2:1b"
    FULL_LLAMA: str = "llama3.2:latest"
    QWEN_3B: str = "qwen2.5:3b"

    # Embeddings
    OLLAMA_EMBEDDING: str = "nomic-embed-text"

In [6]:
class LargeLanguageModel:
    """Central factory for LLMs, chat models, and embeddings."""

    @staticmethod
    def llm(
        model: str = ModelRegistry.TINY_LLAMA,
        temperature: float = 0.1,
        max_tokens: int = 300,
    ) -> OllamaLLM:
        return OllamaLLM(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
        )

    @staticmethod
    def chat(
        model: str = ModelRegistry.QWEN_3B,
        temperature: float = 0.5,
        num_ctx: int = 2048,
    ) -> ChatOllama:
        return ChatOllama(
            model=model,
            temperature=temperature,
            num_ctx=num_ctx,
        )

    @staticmethod
    def embeddings(
        model: str = ModelRegistry.OLLAMA_EMBEDDING,
    ) -> OllamaEmbeddings:
        return OllamaEmbeddings(model=model)

In [7]:
class Chains:
    @staticmethod
    def simple_qa(chat_model: Optional[ChatOllama] = None):
        llm = chat_model or LargeLanguageModel.chat()

        prompt = PromptTemplate(
            input_variables=["question"],
            template="Answer clearly and concisely:\n{question}",
        )

        return prompt | llm | StrOutputParser()

In [15]:
llm = LargeLanguageModel.llm()

In [ ]:
topic = "the life cycle of butterflies"
prompt = PromptTemplate.from_template("""Write an engaging and educational story about {topic} for beginners. 
        Use simple and clear language to explain basic concepts. 
        Include interesting facts and keep it friendly and encouraging. 
        The story should be around 200-300 words and end with a brief summary of what we learned. 
        Make it perfect for someone just starting to learn about this topic.""")

llm = LargeLanguageModel.llm()

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"topic": topic})
print(response)

## Generate-a-story

In [20]:
# Function to generate an educational story using the Mixtral model
def generate_story(topic):
    # Construct a detailed prompt that guides the model to:
    # - Write for beginners
    # - Use simple language
    # - Include interesting facts
    # - Keep a specific length
    # - End with a summary
    prompt = PromptTemplate.from_template("""Write an engaging and educational story about {topic} for beginners. 
            Use simple and clear language to explain basic concepts. 
            Include interesting facts and keep it friendly and encouraging. 
            The story should be around 200-300 words and end with a brief summary of what we learned. 
            Make it perfect for someone just starting to learn about this topic.""")

    llm = LargeLanguageModel.llm()
    
    # Generate text using the model with our carefully crafted prompt
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"topic": topic})
    return response

# Example usage of the generate_story function
# Here we use butterflies as a topic since it's an engaging and 
# educational subject that demonstrates the function well
topic = "the life cycle of butterflies"
story = generate_story(topic)
print("Generated Story:\n", story)

Generated Story:
 In a sunny meadow, a tiny egg cracked open on a leaf. Inside, a hungry caterpillar waited patiently. As the days passed, the caterpillar grew and molted several times, shedding its skin like a changing suit.

One day, the caterpillar attached itself to a leaf and began to spin a silky pad. This was the start of a magical transformation – it was going to become a butterfly! The caterpillar's body broke down into tiny pieces, releasing enzymes that started to break down its own tissues.

As it transformed, the caterpillar's body turned into a chrysalis, a cozy little prison where amazing things were happening. Inside, the caterpillar's body reorganized itself into a completely new form – a butterfly! It was like a puzzle piece fitting together perfectly.

After several weeks or months (depending on the species), the butterfly emerged from its chrysalis. Its wings were soft and folded, but as it pumped blood into them, they expanded and dried. The butterfly's body was no

## Convert-the-story-to-speech

In [ ]:
from gtts import gTTS
from IPython.display import Audio
import io

# Initialize text-to-speech with the generated story
tts = gTTS(story)

# Save the audio to a bytes buffer in memory
audio_bytes = io.BytesIO()
tts.write_to_fp(audio_bytes)
audio_bytes.seek(0)

# Create and display an audio player widget in the notebook
Audio(audio_bytes.read(), autoplay=True)

In [22]:
# Save as MP3 file
tts.save("generated_story.mp3")

In [23]:
topic = "the life cycle of a human"
story = generate_story(topic)

# Initialize text-to-speech with the generated story
tts = gTTS(story)

audio_bytes = io.BytesIO()
tts.write_to_fp(audio_bytes)
audio_bytes.seek(0)

# Create and display an audio player widget in the notebook
Audio(audio_bytes.read(), autoplay=True)